In [1]:
from langchain_community.embeddings import HuggingFaceEmbeddings

# The exact model identifier from Hugging Face
model_name = "intfloat/multilingual-e5-base" 

# Customizing the model to use the instruction-based prefix for retrieval
# The SentenceTransformer utility handles the "passage:" and "query:" prefixes 
# automatically when passed through the HuggingFaceEmbeddings wrapper.
e5_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cpu'}, # Use 'cuda' if you have a GPU
    encode_kwargs={'normalize_embeddings': True} # Essential for Cosine Similarity
)

ModuleNotFoundError: No module named 'langchain_community'

In [2]:
!pip install langchain_community


Defaulting to user installation because normal site-packages is not writeable
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached marshmallow-3.26.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ------------------------------------- -- 2.4/2.5 MB 20.9 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 15.5 MB/s  0:00:00
Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 24.8 MB/s  0:00:00
Using cached marshmallow-3.26.1-py3-none-any.whl (50 kB)
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 2.1/2.1 MB 11.6 MB/s  0:00:00
Using cached typing_inspect-0.9.0-py3-none-any.whl (8.8 kB)

   -- -------------------


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\nniko\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
!pip install sentence_transformers

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
   ----------- ---------------------------- 3.4/12.0 MB 19.6 MB/s eta 0:00:01
   ----------------- ---------------------- 5.2/12.0 MB 13.8 MB/s eta 0:00:01
   ------------------------------- -------- 9.4/12.0 MB 16.1 MB/s eta 0:00:01
   ---------------------------------------- 12.0/12.0 MB 15.3 MB/s  0:00:00
   ---------------------------------------- 0.0/566.1 kB ? eta -:--:--
   ---------------------------------------- 566.1/566.1 kB 19.1 MB/s  0:00:00
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 2.7/2.7 MB 15.5 MB/s  0:00:00
   ---------------------------------------- 0.0/110.9 MB ? eta -:--:--
   - -------------------------------------- 2.9/110.9 MB 16.5 MB/s eta 0:00:07
   - -------------------------------------- 5.0/110.9 MB 13.4 MB/s eta 0:00:08
   -- ---------------


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\nniko\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# 1. Load the E5-Base-Instruct Model
# Use the multilingual-e5-base variant which is instruction-tuned.
model_name = 'intfloat/multilingual-e5-base'
model = SentenceTransformer(model_name)
print(f"Model {model_name} loaded. Embedding dimension: {model.get_sentence_embedding_dimension()}")

# --- Your Data ---
# A. The AI Summary (Query)
ai_summary = "A new chip design using quantum tunneling transistors could boost CPU efficiency by 40%."

# B. Sample Articles (Passages) from your DB
articles = [
    "The latest article on CPU architecture details how Intel's new 10nm process improves power efficiency.", # Related
    "Quantum computing breakthroughs lead to the first stable 50-qubit system, primarily focused on research.", # Somewhat Related
    "A detailed review of the new iPhone model, focusing on camera performance and battery life.", # Unrelated
    "Researchers showcase a quantum-tunneling transistor that dramatically reduces power consumption in microprocessors, achieving up to 40% gains in speed and efficiency." # Highly Related
]

# --- 2. Apply Prefixes and Encode ---

# Prefix the summary as a 'query'
query_text = ["query: " + ai_summary]

# Prefix the articles as 'passages'
passage_texts = ["passage: " + a for a in articles]

# Encode all texts at once
# Use convert_to_tensor=True if you need torch tensors for GPU acceleration, 
# otherwise numpy array (default) is fine for CPU-based operations.
all_embeddings = model.encode(query_text + passage_texts, normalize_embeddings=True)

# Separate the embeddings
summary_embedding = all_embeddings[0]
article_embeddings = all_embeddings[1:]

# The embeddings are typically 768-dimensional vectors (floats)
print(f"Summary Embedding Shape: {summary_embedding.shape}")

C:\Users\nniko\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\nniko\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\nniko\.cache\huggingface\hub\models--intfloat--multilingual-e5-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/hu

Model intfloat/multilingual-e5-base loaded. Embedding dimension: 768
Summary Embedding Shape: (768,)


In [6]:
# The summary vector (a single vector)
V_S = summary_embedding.reshape(1, -1) 

# The matrix of all article vectors (multiple vectors)
V_A = article_embeddings

# Calculate Cosine Similarity between the summary (V_S) and all articles (V_A)
# sklearn's cosine_similarity function handles this matrix-vector operation efficiently.
relevance_scores = cosine_similarity(V_S, V_A)[0]

# --- 3. Display Results ---
results = []
for score, article in zip(relevance_scores, articles):
    # Format the score to two decimal places
    results.append({
        "article": article,
        "relevance_score": round(score, 4) 
    })

# Sort the results by score
ranked_results = sorted(results, key=lambda x: x['relevance_score'], reverse=True)

## Results Table
print("\n--- Relevance Scores (Summary vs. Articles) ---")
print("| Rank | Relevance Score | Article Snippet |")
print("|------|-------------------|-----------------|")
for i, res in enumerate(ranked_results):
    snippet = res['article'][:60] + "..." if len(res['article']) > 60 else res['article']
    print(f"| {i+1}    | **{res['relevance_score']:.4f}** | {snippet} |")


--- Relevance Scores (Summary vs. Articles) ---
| Rank | Relevance Score | Article Snippet |
|------|-------------------|-----------------|
| 1    | **0.9073** | Researchers showcase a quantum-tunneling transistor that dra... |
| 2    | **0.8204** | The latest article on CPU architecture details how Intel's n... |
| 3    | **0.7832** | Quantum computing breakthroughs lead to the first stable 50-... |
| 4    | **0.7513** | A detailed review of the new iPhone model, focusing on camer... |
